In [3]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score,precision_score, recall_score, roc_auc_score
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
import tensorflow.keras
# import keras.metrics
# from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D
# from keras.models import Sequential
# from keras.callbacks import History 
# from keras.utils import plot_model
# from keras.optimizers import SGD
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV, RFE


from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import pandas as pd
import os
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import LeaveOneOut
from imblearn.over_sampling import SMOTE, ADASYN
#from imblearn.pipeline import Pipeline
from xgboost import XGBClassifier

In [4]:
path = "PTSD.xlsx"
df = pd.read_excel(path)
df = df[~df["PCL_Strict3"].isna()]


In [5]:
features = ["T1bias"]

In [6]:

numerical_features = [ "T1Acc1t", "T1Acc1n", "T1bias", "state1", "trait1", "lot1", "phq1",
                      "cd_risc1", "PCL1", "PCL2", "trauma_history8_1"]
categorical_features = ["age", "highschool_diploma",  "dyslexia", "ADHD", "T1Acc1t", "T1Acc1n", "T1bias", "phq1", 
                        "lot1", "trait1", "state1",  "PCL_Broad1", "PCL_Strict1", "active_coping1",
                        "planning1", "positive_reframing1", "acceptance1", "humor1", "religion1", 
                        "emotional_support1","instrumental_support1", "self_distraction1", "denial1", 
                        "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1"]

#df['bad_features'] = (df > df.mean())[bad_features].sum(axis=1)

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
df[numerical_features] = imp.fit_transform(df[numerical_features])

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df[categorical_features] = imp.fit_transform(df[categorical_features])


X = df[features]


# ss = StandardScaler()
# X = ss.fit_transform(X)


Y = df["PCL_Strict3"]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.1, random_state=271828, stratify=Y)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_train, y_train, test_size = 0.1, random_state=271828, stratify=y_train)


In [8]:
pipe =  Pipeline([
    
                ('classifier', XGBClassifier())])

In [9]:
params_grid = [
    {
       # 'pca__n_components': [0.9],
        'classifier__min_child_weight': [1],
        'classifier__gamma': [0],
        'classifier__subsample': [0.9],
        'classifier__colsample_bytree': [0.7],
        'classifier__max_depth': [4, 2],
  #      'classifier__reg_alpha' : [0.2, 0.9],
   #     'classifier__reg_lambda' : [0.5, 0.2, 0.9],
        'classifier__scale_pos_weight' : [1, 2, 0.5],
    'classifier__learning_rate': [0.05, 0.1], #so called `eta` value
              'classifier__n_estimators': [300,100], #number of trees, change it to 1000 for better results
    }]

In [10]:
kfold = StratifiedKFold(n_splits=10, shuffle=True)

gs = GridSearchCV(pipe, params_grid, cv=kfold, scoring='precision')
gs.fit(X_train_2, y_train_2,)

C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=True),
       error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('classifier', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'classifier__min_child_weight': [1], 'classifier__gamma': [0], 'classifier__subsample': [0.9], 'classifier__colsample_bytree': [0.7], 'classifier__max_depth': [4, 2], 'classifier__scale_pos_weight': [1, 2, 0.5], 'classifier__learning_rate': [0.05, 0.1], 'classifier__n_estimators': [300, 100]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_s

In [11]:
gs.best_score_

0.0

In [12]:
gs.best_params_

{'classifier__colsample_bytree': 0.7,
 'classifier__gamma': 0,
 'classifier__learning_rate': 0.05,
 'classifier__max_depth': 4,
 'classifier__min_child_weight': 1,
 'classifier__n_estimators': 300,
 'classifier__scale_pos_weight': 1,
 'classifier__subsample': 0.9}

In [ ]:
X.shape